In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

import pandas as pd 
import numpy as np

In [3]:
browser = Chrome()

In [4]:
url = 'https://www1.ticketmaster.com/event/Z7r9jZ1Aeueuy?routing=y&tmrid=TMR-2342379&tm_link=tm_homeA_header_search'
sel = 'button.landing-modal-footer__see-tickets-button'

In [5]:
browser.get(url)

In [6]:
browser.find_element_by_css_selector(sel).click()

In [7]:
tickets = browser.find_elements_by_css_selector('span.quick-picks__item-desc')

In [8]:
len(tickets)

299

In [17]:
all_tickets = []
for tic in tickets:
    sec = tic.text.split(',')[0].split(' ')[-1]
    row = tic.text.split(', ')[-1].split(' ')[-1]
    curr = {'Section': sec, 'Row': row}
    all_tickets.append(curr)

In [18]:
all_tickets[31]['Row'] = '18'

In [19]:
for ticket in all_tickets:
    ticket['Row'] = int(ticket['Row'])
    ticket['Section'] = int(ticket['Section'])

In [11]:
covered = []
covered.extend(range(201, 212))
covered.extend(range(225, 235))
covered.extend(range(301, 312))
covered.extend(range(325, 334))

In [20]:
for ticket in all_tickets:
    if ticket['Section'] in covered:
        ticket['Covered'] = 1
    else: 
        ticket['Covered'] = 0

In [26]:
beer_rating = {10: [234, 201, 301, 101, 135, 136, 137, 138, 139, 133], 
               9: [333, 233, 302, 202, 132, 102], 
               8: [332, 232, 302, 202, 131, 103], 
               7: [331, 231, 303, 203, 130, 104], 
               6: [330, 230, 304, 204, 129, 105], 
               5: [329, 229, 305, 205, 128, 106], 
               4: [328, 228, 306, 206, 127, 107], 
               3: [327, 227, 307, 207, 126, 108], 
               2: [326, 226, 308, 208, 125, 109], 
               1: [325, 225, 309, 310, 311, 209, 210, 211, 
                   125, 224, 223, 222, 221, 220, 219, 218,
                   217, 216, 215, 214, 213, 212, 122, 121, 
                   120, 119, 118, 117, 116, 115, 115, 110]}

In [27]:
for ticket in all_tickets:
    sec = ticket['Section']
    for key in beer_rating.keys():
        if sec in beer_rating[key]:
            ticket['Beer_Rating'] = key

In [28]:
all_tickets

[{'Beer_Rating': 10, 'Covered': 1, 'Row': 40, 'Section': 301},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 34, 'Section': 311},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 36, 'Section': 311},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 64, 'Section': 325},
 {'Beer_Rating': 8, 'Covered': 1, 'Row': 67, 'Section': 332},
 {'Beer_Rating': 9, 'Covered': 1, 'Row': 63, 'Section': 333},
 {'Beer_Rating': 8, 'Covered': 1, 'Row': 36, 'Section': 302},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 31, 'Section': 311},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 37, 'Section': 310},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 36, 'Section': 325},
 {'Beer_Rating': 9, 'Covered': 1, 'Row': 59, 'Section': 333},
 {'Beer_Rating': 10, 'Covered': 1, 'Row': 30, 'Section': 301},
 {'Beer_Rating': 8, 'Covered': 1, 'Row': 30, 'Section': 302},
 {'Beer_Rating': 1, 'Covered': 1, 'Row': 31, 'Section': 310},
 {'Beer_Rating': 8, 'Covered': 1, 'Row': 65, 'Section': 332},
 {'Beer_Rating': 8, 'Covered': 1, 'Row': 65, 'Section': 332},
 {'Bee

In [53]:
df = pd.DataFrame(all_tickets)

In [30]:
browser = Chrome()

In [31]:
url = 'https://www.rateyourseats.com/tickets/events/arizona-state-sun-devils-at-washington-huskies-september-22-2018-2530478'
browser.get(url)

In [51]:
ticket_ratings = browser.find_elements_by_css_selector('li.ticket')

In [52]:
ticket_ratings = [ticket_ratings[i].text.split('\n') for i in range(len(ticket_ratings))]

In [62]:
ratings = []
for ticket in ticket_ratings:
    sec = ticket[0].split(', ')[0].split(' ')[-1]
    row = ticket[0].split(', ')[1].split(' ')[-1]
    rating = len(ticket[1].split(' | ')[0])
    num_tics = ticket[-3]
    price = int(ticket[-1].split('/')[0][1:])
    curr = {'Section': sec, 'Row': row, 'Rating': rating, 
            'No_Tickets': num_tics, 'Price': price}
    ratings.append(curr)

In [64]:
ratings_df = pd.DataFrame(ratings)

In [67]:
ratings_df.loc[67, 'Row'] = 18

ratings_df['Row'] = ratings_df['Row'].astype(int)

ratings_df['Section'] = ratings_df['Section'].astype(int)

ratings_df['No_Tickets'] = ratings_df['No_Tickets'].apply(lambda x: x.split('-')[-1])

In [77]:
df.merge(ratings_df, on=['Section', 'Row']).to_csv('all_data.csv')